In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np


In [2]:
# Librerías
import pandas as pd
import numpy as np


movies_path = "../ml-1m/movies.dat"
ratings_path = "../ml-1m/ratings.dat"
users_path = "../ml-1m/users.dat"


# Carga de los datos
users = pd.read_csv(users_path, sep="::", engine="python", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], encoding="latin-1")
movies = pd.read_csv(movies_path, sep="::", engine="python", names=["MovieID", "Title", "Genres"], encoding="latin-1")
ratings = pd.read_csv(ratings_path, sep="::", engine="python", names=["UserID", "MovieID", "Rating", "Timestamp"], encoding="latin-1")

# Mostrar primeras filas
ratings.head()



,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
movie_enc = LabelEncoder()

ratings['user'] = user_enc.fit_transform(ratings['UserID'])
ratings['movie'] = movie_enc.fit_transform(ratings['MovieID'])

num_users = ratings['user'].nunique()
num_movies = ratings['movie'].nunique()


In [4]:
# Filtrar películas que están en el dataset de ratings
valid_movie_ids = set(ratings['MovieID'].unique())
movies = movies[movies['MovieID'].isin(valid_movie_ids)].copy()

# Ahora ya no habrá IDs desconocidos
movies['MainGenre'] = movies['Genres'].apply(lambda x: x.split('|')[0])
genre_enc = LabelEncoder()
movies['genre_idx'] = genre_enc.fit_transform(movies['MainGenre'])

# Finalmente puedes codificar los movie IDs correctamente
movie_id_to_genre = dict(zip(movie_enc.transform(movies['MovieID']), movies['genre_idx']))
num_genres = len(genre_enc.classes_)


In [5]:
from collections import defaultdict

max_seq_len = 10  # longitud fija de historial

user_histories = defaultdict(list)
sequence_data = []

# Ordenamos por timestamp
ratings = ratings.sort_values(by=['user', 'Timestamp'])

# Generar secuencias
for row in ratings.itertuples():
    u, m = row.user, row.movie
    hist = user_histories[u][-max_seq_len:]
    if len(hist) >= 1:  # solo si hay al menos un ítem en historial
        padded_hist = [0] * (max_seq_len - len(hist)) + hist  # pad left
        sequence_data.append((padded_hist, m))
    user_histories[u].append(m)


In [6]:
import random 

class GRU4RecNegDatasetWithGenre(Dataset):
    def __init__(self, sequence_data, num_items, movie_to_genre, num_negatives=20):
        self.sequence_data = sequence_data
        self.num_items = num_items
        self.num_negatives = num_negatives
        self.movie_to_genre = movie_to_genre

    def __len__(self):
        return len(self.sequence_data)

    def __getitem__(self, idx):
        seq, pos_item = self.sequence_data[idx]
        neg_items = []
        while len(neg_items) < self.num_negatives:
            neg = random.randint(1, self.num_items - 1)
            if neg != pos_item:
                neg_items.append(neg)

        # Obtener géneros
        pos_genre = self.movie_to_genre.get(pos_item, 0)
        neg_genres = [self.movie_to_genre.get(n, 0) for n in neg_items]

        return (
            torch.tensor(seq, dtype=torch.long),
            torch.tensor(pos_item, dtype=torch.long),
            torch.tensor(pos_genre, dtype=torch.long),
            torch.tensor(neg_items, dtype=torch.long),
            torch.tensor(neg_genres, dtype=torch.long)
        )


In [7]:
from torch.utils.data import random_split, DataLoader

neg_dataset_with_genre = GRU4RecNegDatasetWithGenre(sequence_data, num_items=num_movies, movie_to_genre=movie_id_to_genre, num_negatives=20)

train_size = int(0.8 * len(neg_dataset_with_genre))
val_size = len(neg_dataset_with_genre) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(neg_dataset_with_genre, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)



In [8]:
class GRU4RecRankingWithGenreModel(nn.Module):
    def __init__(self, num_items, num_genres, embedding_dim=64, hidden_dim=128, dropout=0.3):
        super().__init__()
        self.movie_emb = nn.Embedding(num_items, embedding_dim, padding_idx=0)
        self.genre_emb = nn.Embedding(num_genres, 8)  # más pequeño para no sobrecargar

        input_dim = embedding_dim + 8
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True, dropout=dropout)
        self.item_proj = nn.Embedding(num_items, hidden_dim)
        self.genre_proj = nn.Embedding(num_genres, hidden_dim)

    def forward(self, input_seq, input_genres, pos_items, pos_genres, neg_items, neg_genres):
        # Embeddings de entrada
        movie_e = self.movie_emb(input_seq)        # (B, L, D)
        genre_e = self.genre_emb(input_genres)     # (B, L, 8)
        seq_input = torch.cat([movie_e, genre_e], dim=2)  # (B, L, D+8)

        # GRU
        gru_out, _ = self.gru(seq_input)           # (B, L, H)
        user_emb = gru_out[:, -1, :]               # (B, H)

        # Positivo
        pos_e = self.item_proj(pos_items) + self.genre_proj(pos_genres)  # (B, H)
        pos_scores = torch.sum(user_emb * pos_e, dim=1)

        # Negativos
        neg_e = self.item_proj(neg_items) + self.genre_proj(neg_genres)  # (B, N, H)
        user_exp = user_emb.unsqueeze(1).expand_as(neg_e)
        neg_scores = torch.sum(user_exp * neg_e, dim=2)

        return pos_scores, neg_scores


In [18]:
class EarlyStopping:
    def __init__(self, patience=3, delta=1e-4, path='best_model.pth'):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.path = path

    def __call__(self, score, model):
        if self.best_score is None or score > self.best_score + self.delta:
            self.best_score = score
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
            print(f"🔁 No mejora... ({self.counter}/{self.patience})")
            if self.counter >= self.patience:
                print("⏹️ Stop temprano activado")
                self.early_stop = True

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.path)
        print(f"💾 Modelo guardado con mejor score ({self.best_score:.4f})")


In [21]:
def train_gru4rec_ranking_with_genre(model, train_loader, val_loader, epochs=10, lr=0.001, device='cpu', patience=3):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    early_stopping = EarlyStopping(patience=patience, path="best_model.pth")

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for seq, pos_item, pos_genre, neg_items, neg_genres in train_loader:
            seq = seq.to(device)
            pos_item = pos_item.to(device)
            pos_genre = pos_genre.to(device)
            neg_items = neg_items.to(device)
            neg_genres = neg_genres.to(device)

            input_genres = torch.tensor(
                [[movie_id_to_genre.get(i.item(), 0) for i in s] for s in seq],
                dtype=torch.long
            ).to(device)

            optimizer.zero_grad()
            pos_scores, neg_scores = model(seq, input_genres, pos_item, pos_genre, neg_items, neg_genres)

            target_pos = torch.ones_like(pos_scores)
            target_neg = torch.zeros_like(neg_scores)

            loss_pos = criterion(pos_scores, target_pos)
            loss_neg = criterion(neg_scores, target_neg)
            loss = loss_pos + loss_neg.mean()

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"📉 Epoch {epoch+1}: Train Loss = {avg_loss:.4f}")

        # Validar y verificar early stop
        recall = evaluate_gru4rec_recall_at_k(model, val_loader, k=10, device=device)
        early_stopping(recall, model)

        if early_stopping.early_stop:
            break


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
best_model = GRU4RecRankingWithGenreModel(
    num_items=num_movies,
    num_genres=num_genres,
    embedding_dim=study.best_params["embedding_dim"],
    hidden_dim=study.best_params["hidden_dim"],
    dropout=study.best_params["dropout"]
).to(device)

train_gru4rec_ranking_with_genre(
    model=best_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=50,         # ⬅️ más epochs ahora que sabemos los mejores params
    lr=study.best_params["lr"],
    device=device,
    patience=5         # ⬅️ early stopping
)



📉 Epoch 1: Train Loss = 0.7530
Recall@10: 0.0102
💾 Modelo guardado con mejor score (0.0102)
📉 Epoch 2: Train Loss = 0.5771
Recall@10: 0.0105
💾 Modelo guardado con mejor score (0.0105)
📉 Epoch 3: Train Loss = 0.5366
Recall@10: 0.0107
💾 Modelo guardado con mejor score (0.0107)
📉 Epoch 4: Train Loss = 0.5146
Recall@10: 0.0111
💾 Modelo guardado con mejor score (0.0111)
📉 Epoch 5: Train Loss = 0.4999
Recall@10: 0.0114
💾 Modelo guardado con mejor score (0.0114)
📉 Epoch 6: Train Loss = 0.4891
Recall@10: 0.0114
🔁 No mejora... (1/5)
📉 Epoch 7: Train Loss = 0.4813
Recall@10: 0.0121
💾 Modelo guardado con mejor score (0.0121)
📉 Epoch 8: Train Loss = 0.4752
Recall@10: 0.0118
🔁 No mejora... (1/5)
📉 Epoch 9: Train Loss = 0.4706
Recall@10: 0.0116
🔁 No mejora... (2/5)
📉 Epoch 10: Train Loss = 0.4799
Recall@10: 0.0117
🔁 No mejora... (3/5)
📉 Epoch 11: Train Loss = 0.4629
Recall@10: 0.0118
🔁 No mejora... (4/5)
📉 Epoch 12: Train Loss = 0.4622
Recall@10: 0.0123
💾 Modelo guardado con mejor score (0.0123)
📉 E

In [32]:
def evaluate_gru4rec_recall_at_k(model, val_loader, k=10, device='cpu'):
    model.eval()
    # model.to(device)
    hits = 0
    total = 0

    with torch.no_grad():
        for input_seq, pos_item, pos_genre, neg_items, neg_genres in val_loader:
            input_seq = input_seq.to(device)
            pos_item = pos_item.to(device)
            pos_genre = pos_genre.to(device)
            neg_items = neg_items.to(device)
            neg_genres = neg_genres.to(device)

            # 👇 CORREGIDO: construir input_genres correctamente
            input_genres = torch.tensor(
                [[movie_id_to_genre.get(i.item(), 0) for i in s] for s in input_seq],
                dtype=torch.long
            ).to(device)

            emb = model.movie_emb(input_seq)
            genre_emb = model.genre_emb(input_genres)
            seq_input = torch.cat([emb, genre_emb], dim=2)

            gru_out, _ = model.gru(seq_input)
            user_emb = gru_out[:, -1, :]

            all_items = torch.arange(1, model.item_proj.num_embeddings).to(device)
            item_emb = model.item_proj(all_items)
            scores = torch.matmul(user_emb, item_emb.T)
            top_k = torch.topk(scores, k=k, dim=1).indices

            for i in range(pos_item.size(0)):
                if pos_item[i] in top_k[i]:
                    hits += 1
                total += 1


    recall_at_k = hits / total
    print(f"Recall@{k}: {recall_at_k:.4f}")
    return recall_at_k


In [33]:
evaluate_gru4rec_recall_at_k(
    model=ranking_model_with_genre,
    val_loader=val_loader,
    k=10, 
    device=device
)



Recall@10: 0.0026


0.002590100284659565

In [24]:
import optuna

def objective(trial):
    # Hiperparámetros sugeridos por Optuna
    embedding_dim = trial.suggest_categorical("embedding_dim", [32, 64, 128])
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    epochs = 1  # Puedes subirlo cuando todo esté ok

    # Crear modelo con esos hiperparámetros
    model = GRU4RecRankingWithGenreModel(
        num_items=num_movies,
        num_genres=num_genres,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        dropout=dropout
    ).to(device)

    # Entrenar el modelo
    train_gru4rec_ranking_with_genre(
        model=model,
        train_loader=train_loader,
        epochs=epochs,
        lr=lr,
        device=device,
        val_loader=val_loader,
        patience=3
    )

    # Evaluar
    recall = evaluate_gru4rec_recall_at_k(
        model=model,
        val_loader=val_loader,
        k=10,
        device=device
    )

    # Lo que Optuna quiere maximizar
    return recall


In [26]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # puedes aumentar n_trials luego


[I 2025-04-06 15:37:25,493] A new study created in memory with name: no-name-0d0a0ebf-d8c9-479c-9755-bfa61e47bee2
/home/hjasi/Documentos/DeepLearning/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.32596042423861676 and num_layers=1
  warnings.warn(


📉 Epoch 1: Train Loss = 1.0377
Recall@10: 0.0062
💾 Modelo guardado con mejor score (0.0062)


[I 2025-04-06 15:40:56,174] Trial 0 finished with value: 0.006150859510948832 and parameters: {'embedding_dim': 128, 'hidden_dim': 128, 'dropout': 0.32596042423861676, 'lr': 0.0011605404188532965}. Best is trial 0 with value: 0.006150859510948832.


Recall@10: 0.0062


/home/hjasi/Documentos/DeepLearning/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19208948008939122 and num_layers=1
  warnings.warn(


...

Recall@10: 0.0094


/home/hjasi/Documentos/DeepLearning/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.36649041351017636 and num_layers=1
  warnings.warn(


📉 Epoch 1: Train Loss = 0.8266
Recall@10: 0.0085
💾 Modelo guardado con mejor score (0.0085)


[I 2025-04-06 16:45:06,353] Trial 19 finished with value: 0.008549845599847108 and parameters: {'embedding_dim': 32, 'hidden_dim': 256, 'dropout': 0.36649041351017636, 'lr': 0.0028097091251974393}. Best is trial 5 with value: 0.010154198980053713.


Recall@10: 0.0085


In [27]:
print("✨ Mejores hiperparámetros encontrados:")
print(study.best_params)

print("📈 Mejor Recall@10:", study.best_value)


✨ Mejores hiperparámetros encontrados:
{'embedding_dim': 64, 'hidden_dim': 64, 'dropout': 0.3570381059256178, 'lr': 0.009241715028040097}
📈 Mejor Recall@10: 0.010154198980053713


In [28]:
import joblib
joblib.dump(study, "optuna_study.pkl")


['optuna_study.pkl']